In [76]:
## Posterior densities
function posterior_mu(X, xj, y, w, sigma, mu0, tau0)
    
    # Numerator
    return(((sum(xj .* (y - X * w)) / sigma) + (mu0/tau0)) / ((transpose(xj) * xj / sigma) + (1 / tau0)))
    
    end;

function posterior_tau(xj, sigma, tau0)
    
    return(1 / ( (transpose(xj) * (xj ./ sigma)) + (1/tau0)))
    
    end;

function posterior_rate(n, a0)
    
    return((n/2) + a0)
    
    end;

function posterior_scale(X, w, y, b0)
    
    return( (sum((y - (X * w) ).^2) / 2 ) + b0 )
    
    end;

In [74]:
((sum(X[:,2] .* (X * w)) / sigma) + 1/5) / ((transpose(X[:,2]) * X[:,2] / sigma) + (1/5))
((sum(xj .* (X * w)) / sigma) + (mu0/tau0)) / ((transpose(xj) * xj / sigma) + (1 / tau0))

2.611036564911103

In [67]:
X = [2 5 3; 2 3 8; 0 9 8; 1 2 3]
y = [2;5;4;6]
w = [0.9;1.2;1.3]
sigma = 1.2
posterior_tau(X[:,2], 1.2, 25)

0.010079967744103217

In [77]:
posterior_mu(X, X[:,2], y, w, sigma, 1, 5)

-1.9948004025494799

In [ ]:
function gibbs_one_step(X, y, w, sigma, priors)
    
    # Number of params
    nparam = len(priors)
    
    # Number of examples
    n = size(X)[1]
    
    # For each, do
    for j in 1:nparam
        
        # If j == nparam, then sigma
        if j == nparam
            
            sigma = 1 / Gamma(posterior_rate(n, priors[j][:alpha]),
                              posterior_scale(X, w, y, priors[j][:beta]))
        
        # Else, coefficient    
        else
            
            w[j] = Normal(posterior_mu(X[:,1:end .!= j], X[:,j], y, w[1:end .!= j], sigma, 
                                priors[j][:mu], priors[j][:sd]),
                          posterior_tau(xj, sigma, priors[j][:sd]))
            
            end;
        
        end;
    
    # Return the coefficient matrix
    return(
        Dict(
            "w" => w,
            "sigma" => sigma
        )
    )
    
    end;
    
    

In [16]:
A = rand(3,3)

3×3 Array{Float64,2}:
 0.759719  0.395073  0.82096 
 0.742021  0.627641  0.523115
 0.231841  0.834954  0.132468

In [22]:
using Distributions

┌ Info: Recompiling stale cache file /home/jasper/.julia/compiled/v1.0/Distributions/xILW0.ji for Distributions [31c24e10-a181-5473-b8eb-7969acd0382f]
└ @ Base loading.jl:1190


In [42]:
g = Array{Float64}(undef, 10, 5)

10×5 Array{Float64,2}:
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310
 6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310  6.94807e-310

In [35]:
function gibbs_sampler(X, y, initial_values, iterations, priors, burn)
    
    # Open up a results matrix
    res = Array(Float64, (iterations, size(X)[2]))
    
    # For each iterations
    for i in 1:iterations
        
        # One stepp of the gibbs sampler
        w, sigma = gibbs_one_step(X, y, w, sigma, priors)
        
        # Add to results
        res[:,] = [w,sigma]
        
        end;
    
    # Return results
    return(res)
    
    end;